<a href="https://colab.research.google.com/github/dmaresza/tensorflow_course/blob/main/09_TensorFlow_Milestone_Project_2_SkimLit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 2: SkimLit

The purpose of this notebook is to build an NLP model to make reading medical abstracts easier.

The paper we'll be replicating (the source of the dataset that we'll be using) is available here: https://arxiv.org/pdf/1710.06071

Reading through the paper above, we see that the model architecture that was used to achieve the best results is available here: https://arxiv.org/pdf/1612.05251

Much of the work was done in the [lesson notebook](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/video_notebooks/09_SkimLit_nlp_milestone_project_2_video.ipynb), and is continued in this notebook.

### Get data and preprocess it

In [1]:
# Had to install old tensorflow version to make sure code ran properly
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation:

In [2]:
# Installing tensorflow_text for Exercise #5
!pip install tensorflow_text==2.15.0

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
      Successfully uninstalled tf_keras-2.17.0


In [3]:
# Useful imports
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [5]:
# Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import calculate_results

--2024-08-14 15:02:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-14 15:02:01 (94.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
# Download PubMed 200k RCT data from GitHub
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct

# Use the 20k dataset with numbers replaced by '@' sign for the experiments
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25 (from 1)
Receiving objects: 100% (39/39), 177.08 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.


In [7]:
# Function to read and format data before further preprocessing
def preprocess_text_with_line_numbers(filename):
  """
  Returns a list of dictionaries of abstract line data.

  Takes in filename, reads its contents, and sorts through each line,
  extracting things like the target label, the text of the sentence,
  how many sentences are in the current abstract, and what sentence
  number the target line is.
  """
  with open(filename, "r") as f:
    input_lines = f.readlines()
  # input_lines = get_lines(filename) # get all lines from filename
  abstract_lines = "" # create an empty abstract
  abstract_samples = [] # create an empty list of abstracts

  # Loop through each line in the target file
  for line in input_lines:
    if line.startswith("###"): # check to see if the line is an ID line
      abstract_id = line
      abstract_lines = "" # reset the abstract string if the line is an ID line

    elif line.isspace(): # check to see if line is a new line
      abstract_line_split = abstract_lines.splitlines() # split abstract into separate lines

      # Iterate through each line in a single abstract and count them at the same time
      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {} # create an empty dictionary for each line
        target_text_split = abstract_line.split("\t") # split target label from text
        line_data["target"] = target_text_split[0] # get target label
        line_data["text"] = target_text_split[1].lower() # get target text and make it lowercase
        line_data["line_number"] = abstract_line_number # in which position does the line appear within the abstract?
        line_data["total_lines"] = len(abstract_line_split) - 1 # how many total lines are there in the target abstract? (start from 0)
        abstract_samples.append(line_data) # add line data to abstract samples list
    else: # if the above condiditons aren't fulfilled, the line contains a labeled sentence
      abstract_lines += line

  return abstract_samples

In [8]:
# Get data from files and preprocess it
train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt")
test_samples = preprocess_text_with_line_numbers(data_dir + "test.txt")

[{'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients with primary knee oa were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'line_number': 3,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'secondary outcome measures included the western ontari

In [9]:
# Turn data into DataFrames
train_df = pd.DataFrame(train_samples)
val_df = pd.DataFrame(val_samples)
test_df = pd.DataFrame(test_samples)
train_df.head()

,target,text,line_number,total_lines
0,OBJECTIVE,to investigate the efficacy of @ weeks of dail...,0,11
1,METHODS,a total of @ patients with primary knee oa wer...,1,11
2,METHODS,outcome measures included pain reduction and i...,2,11
3,METHODS,pain was assessed using the visual analog pain...,3,11
4,METHODS,secondary outcome measures included the wester...,4,11


In [10]:
# Convert abstract text lines into lists
train_sentences = train_df["text"].tolist()
val_sentences = val_df["text"].tolist()
test_sentences = test_df["text"].tolist()
len(train_sentences), len(val_sentences), len(test_sentences)

(180040, 30212, 30135)

In [11]:
# One-hot encode labels (for training)
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False) # we want a non-sparse matrix
train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_df["target"].to_numpy().reshape(-1, 1))
test_labels_one_hot = one_hot_encoder.transform(test_df["target"].to_numpy().reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# Label encode labels (for prediction metrics)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(val_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test_df["target"].to_numpy())

In [13]:
# Get class names and number of classes
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes, class_names

(5,
 array(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'],
       dtype=object))

## 1. Train `model_5` on all of the data in the training dataset for as many epochs until it stops improving.
Since this might take a while, you might want to use:
* `tf.keras.callbacks.ModelCheckpoint` to save the model's best weights only
* `tf.keras.callbacks.EarlyStopping` to stop the model from training once the validation loss has stopped improving for ~3 epochs.

### Turn train and validation data into tf.data Datasets for training

In [109]:
# Split sequence-level data splits into character-level data splits
train_chars = [" ".join(list(sentence)) for sentence in train_sentences]
val_chars = [" ".join(list(sentence)) for sentence in val_sentences]

# Create one-hot-encoded tensors of the "line_number" column
train_line_numbers = tf.one_hot(train_df["line_number"].to_numpy(), depth=15)
val_line_numbers = tf.one_hot(val_df["line_number"].to_numpy(), depth=15)

# Create one-hot-encoded tensors of the "total_lines" feature
train_total_lines = tf.one_hot(train_df["total_lines"].to_numpy(), depth=20)
val_total_lines = tf.one_hot(val_df["total_lines"].to_numpy(), depth=20)

# Create train dataset (with all four kinds of input data)
train_data = tf.data.Dataset.from_tensor_slices((train_line_numbers,
                                                 train_total_lines,
                                                 train_sentences,
                                                 train_chars))
train_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_data, train_labels))
train_dataset = train_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

# Do the same as above but for the validation dataset
val_data = tf.data.Dataset.from_tensor_slices((val_line_numbers,
                                               val_total_lines,
                                               val_sentences,
                                               val_chars))
val_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_data, val_labels))
val_dataset = val_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

In [15]:
# Download saved version of model_5
!wget https://storage.googleapis.com/ztm_tf_course/skimlit/skimlit_tribrid_model.zip
!mkdir skimlit_gs_model
!unzip skimlit_tribrid_model.zip -d skimlit_gs_model
!rm skimlit_tribrid_model.zip

--2024-08-14 15:02:22--  https://storage.googleapis.com/ztm_tf_course/skimlit/skimlit_tribrid_model.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 962182847 (918M) [application/zip]
Saving to: ‘skimlit_tribrid_model.zip’

skimlit_tribrid_mod 100%[===================>] 917.61M  25.0MB/s    in 36s     

2024-08-14 15:02:58 (25.6 MB/s) - ‘skimlit_tribrid_model.zip’ saved [962182847/962182847]

Archive:  skimlit_tribrid_model.zip
   creating: skimlit_gs_model/skimlit_tribrid_model/
  inflating: skimlit_gs_model/skimlit_tribrid_model/keras_metadata.pb  
   creating: skimlit_gs_model/skimlit_tribrid_model/assets/
 extracting: skimlit_gs_model/skimlit_tribrid_model/fingerprint.pb  
   creating: skimlit_gs_model/skimlit_tribrid_model/variables/
  inflating: skimlit_gs_model/s

In [16]:
# Load in downloaded model
model_5 = tf.keras.models.load_model("skimlit_gs_model/skimlit_tribrid_model/")
model_5.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 char_inputs (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 token_inputs (InputLayer)   [(None,)]                    0         []                            
                                                                                                  
 char_vectorizer (TextVecto  (None, 290)                  0         ['char_inputs[0][0]']         
 rization)                                                                                        
                                                                                                  
 universal_sentence_encoder  (None, 512)                  2567978   ['token_inputs[0][0]']  

In [22]:
# Create ModelCheckpoint callback
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="model_5",
                                                      monitor="val_loss",
                                                      save_weights_only=True,
                                                      save_best_only=True)

# Create EarlyStopping callback
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                              patience=3,
                                              restore_best_weights=True)

In [23]:
# Compile the model
model_5.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_5_history = model_5.fit(train_dataset,
                              epochs=100,
                              validation_data=val_dataset,
                              callbacks=[model_checkpoint,
                                         early_stop])

Epoch 1/100
352/352 [==============================] - 64s 142ms/step - loss: 0.4351 - accuracy: 0.8390 - val_loss: 0.3930 - val_accuracy: 0.8497
Epoch 2/100
352/352 [==============================] - 42s 118ms/step - loss: 0.3858 - accuracy: 0.8547 - val_loss: 0.3701 - val_accuracy: 0.8574
Epoch 3/100
352/352 [==============================] - 50s 141ms/step - loss: 0.3628 - accuracy: 0.8638 - val_loss: 0.3573 - val_accuracy: 0.8611
Epoch 4/100
352/352 [==============================] - 49s 138ms/step - loss: 0.3459 - accuracy: 0.8703 - val_loss: 0.3507 - val_accuracy: 0.8645
Epoch 5/100
352/352 [==============================] - 52s 147ms/step - loss: 0.3325 - accuracy: 0.8754 - val_loss: 0.3461 - val_accuracy: 0.8662
Epoch 6/100
352/352 [==============================] - 49s 140ms/step - loss: 0.3215 - accuracy: 0.8800 - val_loss: 0.3445 - val_accuracy: 0.8674
Epoch 7/100
352/352 [==============================] - 50s 141ms/step - loss: 0.3120 - accuracy: 0.8842 - val_loss: 0.3435 -

In [24]:
# Make predictions with the model
model_5_pred_probs = model_5.predict(val_dataset, verbose=1)

# Convert pred probs to pred labels
model_5_preds = tf.argmax(model_5_pred_probs, axis=1)

# Calculate results of model_5
model_5_val_results = calculate_results(y_true=val_labels_encoded,
                                        y_pred=model_5_preds)
model_5_val_results

60/60 [==============================] - 6s 73ms/step


{'accuracy': 86.92241493446313,
 'precision': 0.8733841500780213,
 'recall': 0.8692241493446313,
 'f1': 0.8663665483557124}

When trained on all of the data, the model was able to achieve nearly 87% accuracy on the validation dataset, compared to ~83% accuracy when trained on only 10% of the data.

## 2. Turn the test data samples into a tf.data Dataset (fast loading) and then evaluate the best performing model (`model_5`) on the test samples.

In [110]:
# Split sequence-level data split into character-level data split
test_chars = [" ".join(list(sentence)) for sentence in test_sentences]

# Create one-hot-encoded tensors of the "line_number" column
test_line_numbers = tf.one_hot(test_df["line_number"].to_numpy(), depth=15)

# Create one-hot-encoded tensors of the "total_lines" feature
test_total_lines = tf.one_hot(test_df["total_lines"].to_numpy(), depth=20)

# Create test dataset (with all four kinds of input data)
test_data = tf.data.Dataset.from_tensor_slices((test_line_numbers,
                                                test_total_lines,
                                                test_sentences,
                                                test_chars))
test_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_data, test_labels))
test_dataset = test_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

In [26]:
# Evaluate model_5 on the test data
model_5.evaluate(test_dataset)

59/59 [==============================] - 4s 63ms/step - loss: 0.3621 - accuracy: 0.8630


[0.3620966970920563, 0.862983226776123]

In [27]:
# Make predictions with the model
model_5_pred_probs = model_5.predict(test_dataset, verbose=1)

# Convert pred probs to pred labels
model_5_preds = tf.argmax(model_5_pred_probs, axis=1)

# Calculate results of model_5
model_5_test_results = calculate_results(y_true=test_labels_encoded,
                                         y_pred=model_5_preds)
model_5_test_results

59/59 [==============================] - 5s 82ms/step


{'accuracy': 86.29832420773187,
 'precision': 0.8658495260268554,
 'recall': 0.8629832420773187,
 'f1': 0.8601149553742251}

Predicting on the test dataset gives very similar results to predicting on the validation dataset, although for some reason it seems that the validation dataset yields slightly better results.

## 3. Find the most wrong predictions from exercise 2 (these are the samples where the model has predicted the wrong label with the highest prediction probability).

In [91]:
model_5_pred_probs = model_5.predict(test_dataset)
model_5_preds = tf.argmax(model_5_pred_probs, axis=1)
pred_probs = model_5_pred_probs.max(axis=1)
model_5_labels = [class_names[i] for i in model_5_preds]

# Create DataFrame with test sentences, target labels, predicted labels, and prediction probabilities
test_preds_df = pd.DataFrame({"text": test_df["text"],
                              "target": test_df["target"],
                              "pred": model_5_labels,
                              "pred_prob": pred_probs})
test_preds_df.head()

59/59 [==============================] - 4s 64ms/step


,text,target,pred,pred_prob
0,this study analyzed liver function abnormaliti...,BACKGROUND,OBJECTIVE,0.511526
1,a post hoc analysis was conducted with the use...,RESULTS,METHODS,0.698331
2,liver function tests ( lfts ) were measured at...,RESULTS,METHODS,0.988842
3,survival analyses were used to assess the asso...,RESULTS,METHODS,0.966783
4,the percentage of patients with abnormal lfts ...,RESULTS,RESULTS,0.976663


In [92]:
# 3. Find out which predictions are wrong in our DataFrame
test_preds_df["pred_correct"] = test_preds_df["target"] == test_preds_df["pred"]

In [120]:
# Create new DataFrame of 100 most wrong predictions
top_100_wrong = test_preds_df[test_preds_df["pred_correct"] == False].sort_values("pred_prob", ascending=False)[:100]
top_100_wrong.head(10)

,text,target,pred,pred_prob,pred_correct
13598,-@ % vs. fish : -@ % vs. fish + s : -@ % ; p <...,METHODS,RESULTS,0.999860,False
1827,nct@ ( clinicaltrials.gov ) .,CONCLUSIONS,BACKGROUND,0.999752,False
25664,rifampicin significantly increased the mean ar...,CONCLUSIONS,RESULTS,0.999552,False
3573,a cluster randomised trial was implemented wit...,RESULTS,METHODS,0.999443,False
9468,pdt was associated with a significant decrease...,CONCLUSIONS,RESULTS,0.999039,False
12269,patients received oral se tablets ( @ mcg ) or...,RESULTS,METHODS,0.998975,False
29097,the primary endpoint was change from baseline ...,BACKGROUND,METHODS,0.998919,False
24512,"in contrast , displayed values changed rapidly...",METHODS,RESULTS,0.998911,False
12158,"in the saffron group , there was no decline in...",METHODS,RESULTS,0.998836,False
23202,the prebious trial has the potential to demons...,RESULTS,CONCLUSIONS,0.998668,False


## 4. Check out the [Keras guide on using pretrained GloVe embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/). Can you get this working with one of our models?
* Hint: You'll want to incorporate it with a custom token [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.
* It's up to you whether or not you fine-tune the GloVe embeddings or leave them frozen.

In [94]:
# Downloading pretrained GloVe embeddings
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-08-14 19:03:20--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.14MB/s    in 2m 40s  

2024-08-14 19:06:02 (5.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [95]:
# Using the 300-dimensional embeddings to best replicate the paper
path_to_glove_file = "glove.6B.300d.txt"

# Make a dict mapping words to their numpy vector representations
embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [96]:
# Create a text vectorization layer
text_vectorizer = layers.TextVectorization(max_tokens=68000, # 68k vocab size from paper
                                           output_sequence_length=55) # 55-word length covers 95% of examples

# Adapt text vectorizer to training sentences
text_vectorizer.adapt(train_sentences)

In [97]:
# Get the total vocabulary of the vectorizer
text_vocab = text_vectorizer.get_vocabulary()

# Create dict mapping words to their indices
word_index = dict(zip(text_vocab, range(len(text_vocab))))

In [98]:
# Preparing an embedding matrix to use with a Keras Embedding layer
num_tokens = len(text_vocab) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # Words not found in embedding index will be all-zeros.
    # This includes the representation for "padding" and "OOV"
    embedding_matrix[i] = embedding_vector
    hits += 1
  else:
    misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 29730 words (35111 misses)


In [99]:
# Create custom Embedding layer
glove_embedding_layer = layers.Embedding(input_dim=num_tokens,
                                         output_dim=embedding_dim,
                                         embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                         trainable=False)

In [100]:
# Test out Embedding layer on random sentence
import random

target_sentence = random.choice(test_sentences)
vectorized_sentence = text_vectorizer([target_sentence])
embedded_sentence = glove_embedding_layer(vectorized_sentence)
print(f"Sentence before vectorization:\n {target_sentence}\n")
print(f"Sentence after vectorization (before embedding):\n{vectorized_sentence}\n")
print(f"Sentence after embedding:\n{embedded_sentence}\n")
print(f"Embedded sentence shape: {embedded_sentence.shape}")

Sentence before vectorization:
 australian new zealand clinical trials registry actrn@ .

Sentence after vectorization (before embedding):
[[1561  319 1691   47  233 1150 1776    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]]

Sentence after embedding:
[[[-0.020175  0.21116  -0.15082  ...  0.39348   0.10561  -0.39483 ]
  [-0.62333  -0.42434  -0.035321 ... -0.13613   0.09868   0.609   ]
  [-0.21495   0.084593 -0.29487  ...  0.23793   0.48038  -0.33481 ]
  ...
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]]]

Embedded sentence shape: (1, 55, 300)


In [113]:
# Create model using custom embedding layer
inputs = layers.Input(shape=(None,), dtype="int32")
token_embeddings = glove_embedding_layer(inputs)
x = layers.Conv1D(128, 5, activation="relu", padding="same")(token_embeddings)
x = layers.MaxPooling1D(5, padding="same")(x)
x = layers.Conv1D(128, 5, activation="relu", padding="same")(x)
x = layers.MaxPooling1D(5, padding="same")(x)
x = layers.Conv1D(128, 5, activation="relu", padding="same")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

glove_model = tf.keras.Model(inputs, outputs)
glove_model.summary()

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       multiple                  19452900  
                                                                 
 conv1d_6 (Conv1D)           (None, None, 128)         192128    
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                     

In [114]:
train_sen_vectors = text_vectorizer(np.array([[sen] for sen in train_sentences])).numpy()
val_sen_vectors = text_vectorizer(np.array([[sen] for sen in val_sentences])).numpy()
test_sen_vectors = text_vectorizer(np.array([[sen] for sen in test_sentences])).numpy()

# Create datasets for the model
train_dataset = tf.data.Dataset.from_tensor_slices((train_sen_vectors, train_labels_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_sen_vectors, val_labels_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_sen_vectors, test_labels_encoded)).batch(32).prefetch(tf.data.AUTOTUNE)

In [115]:
# Compile the model
glove_model.compile(loss="sparse_categorical_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])

# Fit the model
glove_model_history = glove_model.fit(train_dataset,
                                      epochs=100,
                                      validation_data=val_dataset,
                                      callbacks=[early_stop])

Epoch 1/100
5627/5627 [==============================] - 49s 9ms/step - loss: 0.6214 - accuracy: 0.7717 - val_loss: 0.5099 - val_accuracy: 0.8125
Epoch 2/100
5627/5627 [==============================] - 37s 7ms/step - loss: 0.5011 - accuracy: 0.8187 - val_loss: 0.5075 - val_accuracy: 0.8163
Epoch 3/100
5627/5627 [==============================] - 45s 8ms/step - loss: 0.4493 - accuracy: 0.8367 - val_loss: 0.5305 - val_accuracy: 0.8168
Epoch 4/100
5627/5627 [==============================] - 39s 7ms/step - loss: 0.4061 - accuracy: 0.8519 - val_loss: 0.5792 - val_accuracy: 0.8129
Epoch 5/100
5627/5627 [==============================] - 42s 7ms/step - loss: 0.3736 - accuracy: 0.8641 - val_loss: 0.6309 - val_accuracy: 0.8055


In [118]:
# Evaluate on the test dataset
glove_model.evaluate(test_dataset)

942/942 [==============================] - 4s 4ms/step - loss: 0.5260 - accuracy: 0.8094


[0.5259901285171509, 0.8093578815460205]

In [119]:
# Make predictions with the model
glove_model_pred_probs = glove_model.predict(test_dataset, verbose=1)

# Convert pred probs to pred labels
glove_model_preds = tf.argmax(glove_model_pred_probs, axis=1)

# Calculate results of model_5
glove_model_results = calculate_results(y_true=test_labels_encoded,
                                        y_pred=glove_model_preds)
glove_model_results

942/942 [==============================] - 2s 2ms/step


{'accuracy': 80.93578894972623,
 'precision': 0.8106525423110673,
 'recall': 0.8093578894972623,
 'f1': 0.8056769367134935}

I created a simpler model for the GloVe embeddings, and this Conv1D model achieved a respectable ~81% accuracy on the test dataset, compared to ~86.3% for the model in Exercise #1.

## 5. Try replacing the TensorFlow Hub Universal Sentence Encoder pretrained embedding with the [TensorFlow Hub BERT PubMed expert](https://www.kaggle.com/models/google/experts-bert/tensorFlow2/pubmed/2?tfhub-redirect=true) pretrained embedding (a language model pretrained on PubMed texts). Does this affect the results?
* Note: Using the BERT PubMed expert pretrained embedding requires an extra preprocessing step for sequences (as detailed in the [TensorFlow Hub guide](https://www.kaggle.com/models/google/experts-bert/tensorFlow2/pubmed/2?tfhub-redirect=true)).
* Does the BERT model beat the results mentioned in [this paper](https://arxiv.org/pdf/1710.06071.pdf)?

In [44]:
# Import preprocessing and encoder models
import tensorflow_text as text
import tensorflow_hub as hub

preprocessing_layer = hub.KerasLayer('https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3',
                                     trainable=False, name="pubmed_bert_preprocessor")

bert_layer = hub.KerasLayer('https://www.kaggle.com/models/google/experts-bert/TensorFlow2/pubmed/2',
                            trainable=False, name="bert_model_layer")

In [45]:
# Get all keyboard characters
import string

alphabet = string.ascii_lowercase + string.digits + string.punctuation
alphabet

'abcdefghijklmnopqrstuvwxyz0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [46]:
# Create char-level token vectorizer instance
NUM_CHAR_TOKENS = len(alphabet) + 2 # add 2 for space and OOV token (OOV = out of vocab, '[UNK]')
char_vectorizer = layers.TextVectorization(max_tokens=NUM_CHAR_TOKENS,
                                           output_sequence_length=290, # 290 characters covers 95th percentile of sentence lengths
                                           name="char_vectorizer")

# Adapt character vectorizer to training characters
char_vectorizer.adapt(train_chars)

In [47]:
# Create character embedding layer
char_embed = layers.Embedding(input_dim=len(char_vectorizer.get_vocabulary()), # number of different characters
                              output_dim=25, # this is the size of the char embedding in the paper (Figure 1)
                              mask_zero=True,
                              name="char_embed")

In [48]:
# Recreate model_5 structure with BERT embedding instead of USE
# 1. Token inputs
token_inputs = layers.Input(shape=[], dtype="string", name="token_inputs")
bert_inputs = preprocessing_layer(token_inputs)
bert_embedding = bert_layer(bert_inputs, training=False)
token_outputs = layers.Dense(128, activation="relu")(bert_embedding["pooled_output"])
token_model = tf.keras.Model(inputs=token_inputs, outputs=token_outputs)

# 2. Char inputs
char_inputs = layers.Input(shape=[], dtype="string", name="char_inputs")
bert_char_inputs = preprocessing_layer(char_inputs)
bert_char_embedding = bert_layer(bert_char_inputs, training=False)
char_outputs = layers.Dense(128, activation="relu")(bert_char_embedding["pooled_output"])
char_model = tf.keras.Model(inputs=char_inputs, outputs=char_outputs)

# 3. Line numbers model
line_number_inputs = layers.Input(shape=(15,), dtype=tf.float32, name="line_number_input")
# dense layer with 32 units & relu activation
x = layers.Dense(32, activation="relu")(line_number_inputs)
# combine inputs & dense layer into model
line_number_model = tf.keras.Model(inputs=line_number_inputs, outputs=x)

# 4. Total lines model
total_lines_inputs = layers.Input(shape=(20,), dtype=tf.float32, name="total_lines_input")
y = layers.Dense(32, activation="relu")(total_lines_inputs)
total_lines_model = tf.keras.Model(inputs=total_lines_inputs, outputs=y)

# 5. Combine token and char embeddings into a hybrid embedding
combined_embeddings = layers.Concatenate(name="char_token_hybrid_embedding")([token_model.output,
                                                                              char_model.output])

z = layers.Dense(256, activation="relu")(combined_embeddings)
z = layers.Dropout(0.5)(z)

# 6. Combine positional embeddings with combined token and char embeddings
tribrid_embeddings = layers.Concatenate(name="char_token_positional_embedding")([line_number_model.output,
                                                                                total_lines_model.output,
                                                                                z])

# 7. Create output layer
output_layer = layers.Dense(num_classes, activation="softmax", name="output_layer")(tribrid_embeddings)

# 8. Put together model with all kinds of inputs
bert_model = tf.keras.Model(inputs=[line_number_model.input,
                                    total_lines_model.input,
                                    token_model.input,
                                    char_model.input],
                            outputs=output_layer,
                            name="bert_tribrid_embedding_model")

bert_model.summary()

Model: "bert_tribrid_embedding_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 token_inputs (InputLayer)   [(None,)]                    0         []                            
                                                                                                  
 char_inputs (InputLayer)    [(None,)]                    0         []                            
                                                                                                  
 pubmed_bert_preprocessor (  {'input_mask': (None, 128)   0         ['token_inputs[0][0]',        
 KerasLayer)                 , 'input_type_ids': (None,              'char_inputs[0][0]']         
                              128),                                                               
                              'input_word_ids': (None,                 

In [50]:
# Compile the model
bert_model.compile(loss="categorical_crossentropy",
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=["accuracy"])

# Train model on 10% of data for faster training times
bert_model_history = bert_model.fit(train_dataset,
                                    steps_per_epoch=int(0.1*len(train_dataset)),
                                    epochs=3,
                                    validation_data=val_dataset,
                                    validation_steps=int(0.1*len(val_dataset)))

Epoch 1/3
35/35 [==============================] - 952s 27s/step - loss: 0.8931 - accuracy: 0.6752 - val_loss: 0.5198 - val_accuracy: 0.8268
Epoch 2/3
35/35 [==============================] - 926s 27s/step - loss: 0.5107 - accuracy: 0.8243 - val_loss: 0.4454 - val_accuracy: 0.8415
Epoch 3/3
35/35 [==============================] - 930s 27s/step - loss: 0.4598 - accuracy: 0.8399 - val_loss: 0.4028 - val_accuracy: 0.8493


In [51]:
# Evaluate on test dataset
bert_model.evaluate(test_dataset)

59/59 [==============================] - 1337s 23s/step - loss: 0.4225 - accuracy: 0.8491


[0.42246124148368835, 0.8490791320800781]

In [52]:
# Make predictions with the model
bert_model_pred_probs = bert_model.predict(test_dataset, verbose=1)

# Convert pred probs to pred labels
bert_model_preds = tf.argmax(bert_model_pred_probs, axis=1)

# Calculate results of model_5
bert_model_results = calculate_results(y_true=test_labels_encoded,
                                       y_pred=bert_model_preds)
bert_model_results

59/59 [==============================] - 1328s 23s/step


{'accuracy': 84.90791438526631,
 'precision': 0.847216064982828,
 'recall': 0.849079143852663,
 'f1': 0.847004367308248}

I only trained the BERT model for 3 epochs on 10% of the data because it took SO incredibly long to train. With only a few epochs and limited data this model was able to achieve an F1-score of ~84.7% on the test dataset, which is not as high as the 90% achieved in [this paper](https://arxiv.org/pdf/1710.06071), but it could have improved more if I was able to run the model for longer.

## 6. What happens if you merge the `line_number` and `total_lines` features for each sequence? For example, if you created an `X_of_Y` feature instead? Does this affect model performance?
* Another example: `line_number=1` and `total_lines=11` turns into `line_of_X=1_of_11`.

In [ ]:
# Combining line numbers and total lines into new DataFrame column
train_df["line_number_total"] = train_df["line_number"].astype(str) + "_of_" + train_df["total_lines"].astype(str)
val_df["line_number_total"] = val_df["line_number"].astype(str) + "_of_" + val_df["total_lines"].astype(str)
test_df["line_number_total"] = test_df["line_number"].astype(str) + "_of_" + test_df["total_lines"].astype(str)

train_df.head()

In [54]:
# Perform one hot encoding on the train and validation DataFrames
one_hot_encoder = OneHotEncoder()

# Fitting on the training DataFrame
one_hot_encoder.fit(np.expand_dims(train_df["line_number_total"], axis=1))

# Transforming both train and val df
train_line_number_total_encoded = one_hot_encoder.transform(np.expand_dims(train_df["line_number_total"], axis=1))
val_line_number_total_encoded  = one_hot_encoder.transform(np.expand_dims(val_df["line_number_total"], axis=1))
test_line_number_total_encoded = one_hot_encoder.transform(np.expand_dims(test_df["line_number_total"], axis=1))

# Checking the shapes
train_line_number_total_encoded.shape, val_line_number_total_encoded.shape, test_line_number_total_encoded.shape

((180040, 460), (30212, 460), (30135, 460))

In [55]:
# Converting the sparse object to array
train_line_number_total_encoded = train_line_number_total_encoded.toarray()
val_line_number_total_encoded = val_line_number_total_encoded.toarray()
test_line_number_total_encoded = test_line_number_total_encoded.toarray()

# Converting the datatype to int
train_line_number_total_encoded = tf.cast(train_line_number_total_encoded, dtype=tf.int32)
val_line_number_total_encoded = tf.cast(val_line_number_total_encoded, dtype=tf.int32)
test_line_number_total_encoded = tf.cast(test_line_number_total_encoded, dtype=tf.int32)

In [57]:
# Making the performant datasets for our tribid model
train_data = tf.data.Dataset.from_tensor_slices((train_sentences,
                                                 train_chars,
                                                 train_line_number_total_encoded))

train_labels = tf.data.Dataset.from_tensor_slices(train_labels_encoded)

val_data = tf.data.Dataset.from_tensor_slices((val_sentences,
                                               val_chars,
                                               val_line_number_total_encoded))

val_labels = tf.data.Dataset.from_tensor_slices(val_labels_encoded)

test_data = tf.data.Dataset.from_tensor_slices((test_sentences,
                                                test_chars,
                                                test_line_number_total_encoded))

test_labels = tf.data.Dataset.from_tensor_slices(test_labels_encoded)

# Zipping the data and labels
train_dataset = tf.data.Dataset.zip((train_data, train_labels))
val_dataset = tf.data.Dataset.zip((val_data, val_labels))
test_dataset = tf.data.Dataset.zip((test_data, test_labels))

# Applying batch and prefetching
train_dataset = train_dataset.batch(512).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(512).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

train_dataset, val_dataset, test_dataset

(<_PrefetchDataset element_spec=((TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 460), dtype=tf.int32, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=((TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 460), dtype=tf.int32, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=((TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 460), dtype=tf.int32, name=None)), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [60]:
# Buidling the tribid model using the functional api
input_token = layers.Input(shape=[], dtype=tf.string)
bert_inputs_token = preprocessing_layer(input_token)
bert_embedding_char =bert_layer(bert_inputs_token)
output_token = layers.Dense(64, activation="relu")(bert_embedding_char["pooled_output"])
token_model = tf.keras.Model(input_token, output_token)

input_char = layers.Input(shape=[], dtype=tf.string)
bert_inputs_char = preprocessing_layer(input_char)
bert_embedding_char =bert_layer(bert_inputs_char)
output_char = layers.Dense(64, activation="relu")(bert_embedding_char["pooled_output"])
char_model = tf.keras.Model(input_char, output_char)

line_number_total_input = layers.Input(shape=(460,), dtype=tf.int32)
dense = layers.Dense(32, activation="relu")(line_number_total_input)
total_line_number_model = tf.keras.Model(line_number_total_input, dense)

# Concatenating the tokens amd chars output (Hybrid!!!)
combined_embeddings = layers.Concatenate(name="token_char_hybrid_embedding")([token_model.output,
                                                                              char_model.output])

# Combining the line_number_total to our hybrid model (Time for Tribid!!)
z = layers.Concatenate(name="tribid_embeddings")([total_line_number_model.output,
                                                  combined_embeddings])

# Adding a dense + dropout and creating our output layer
dropout = layers.Dropout(0.5)(z)
x = layers.Dense(128, activation="relu")(dropout)
output_layer = layers.Dense(5, activation="softmax")(x)

# Packing into a model
tribrid_model = tf.keras.Model(inputs=[token_model.input,
                                       char_model.input,
                                       total_line_number_model.input],
                               outputs=output_layer)

tribrid_model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 pubmed_bert_preprocessor (  {'input_mask': (None, 128)   0         ['input_4[0][0]',             
 KerasLayer)                 , 'input_type_ids': (None,              'input_5[0][0]']             
                              128),                                                               
                              'input_word_ids': (None,                                     

In [61]:
# Compile the model
tribrid_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

# Fit the model
tribrid_model_history = tribrid_model.fit(train_dataset,
                                          steps_per_epoch=int(0.1*len(train_dataset)),
                                          epochs=3,
                                          validation_data=val_dataset,
                                          validation_steps=int(0.1*len(val_dataset)))

Epoch 1/3
35/35 [==============================] - 962s 27s/step - loss: 1.0456 - accuracy: 0.6059 - val_loss: 0.6380 - val_accuracy: 0.7822
Epoch 2/3
35/35 [==============================] - 943s 27s/step - loss: 0.6245 - accuracy: 0.7848 - val_loss: 0.4720 - val_accuracy: 0.8379
Epoch 3/3
35/35 [==============================] - 934s 27s/step - loss: 0.5310 - accuracy: 0.8141 - val_loss: 0.4273 - val_accuracy: 0.8464


In [63]:
# Evaluate the model
tribrid_model.evaluate(test_dataset)

59/59 [==============================] - 1347s 23s/step - loss: 0.4481 - accuracy: 0.8382


[0.4480772912502289, 0.8381615877151489]

In [78]:
# Make predictions with the model
tribrid_model_pred_probs = tribrid_model.predict(test_dataset, verbose=1)

# Convert pred probs to pred labels
tribrid_model_preds = tf.argmax(tribrid_model_pred_probs, axis=1)

# Calculate results of model_5
tribrid_model_results = calculate_results(y_true=test_labels_encoded,
                                          y_pred=tribrid_model_preds)
tribrid_model_results

59/59 [==============================] - 1373s 23s/step


{'accuracy': 83.8161606105857,
 'precision': 0.8376790556343388,
 'recall': 0.838161606105857,
 'f1': 0.8362848053097415}

Merging the `line_number` and `total_lines` features into a single feature seems to have made the model perform slighly worse than keeping them separate, although the difference is only about 1% lower accuracy.

## 7. Write a function (or series of functions) to take a sample abstract string, preprocess it (in the same way our model has been trained), make a prediction on each sequence in the abstract, and return the abstract in the format:
* `PREDICTED_LABEL`: `SEQUENCE`
* `PREDICTED_LABEL`: `SEQUENCE`
* `PREDICTED_LABEL`: `SEQUENCE`
* `PREDICTED_LABEL`: `SEQUENCE`
* ...
  * You can find your own unstructured RCT abstract from [PubMed](https://pubmed.ncbi.nlm.nih.gov/) or try [this one](https://pubmed.ncbi.nlm.nih.gov/22244707/), or [these](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/skimlit_example_abstracts.json).

In [64]:
# Get some example abstracts from GitHub
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/skimlit_example_abstracts.json

--2024-08-14 18:01:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/skimlit_example_abstracts.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6737 (6.6K) [text/plain]
Saving to: ‘skimlit_example_abstracts.json’

skimlit_example_abs 100%[===================>]   6.58K  --.-KB/s    in 0s      

2024-08-14 18:01:13 (86.9 MB/s) - ‘skimlit_example_abstracts.json’ saved [6737/6737]



In [65]:
# Use json to load in the samples
import json
with open("skimlit_example_abstracts.json", "r") as f:
  example_abstracts = json.load(f)

example_abstracts

[{'abstract': 'This RCT examined the efficacy of a manualized social intervention for children with HFASDs. Participants were randomly assigned to treatment or wait-list conditions. Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language. A response-cost program was applied to reduce problem behaviors and foster skills acquisition. Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures). Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents. High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity. Standardized effect size estimates were primarily in the medium and large ranges and favored the treatment group.',
  'source': 'https://pubmed.ncbi.nlm.nih.gov/20232240/',
  'details': 'RCT of a manualiz

In [66]:
# Create DataFrame
sample_abstracts = pd.DataFrame(example_abstracts)
sample_abstracts.head()

In [122]:
# Create the function
from spacy.lang.en import English

def predict_on_abstract(abstract_dict, model, label_encoder):
  '''

    Takes in a list of dictionaries of abstracts,

    [{'abstract': 'This RCT examined .......' ,
      'details': 'RCT of a manuali......',
      'source': 'https://pubmed.ncbi.nlm........./'},..........]

    Arguments:
    ----------
      - abstract_dict : Abstract dictionary of the above format
      - model : the trained model on the same data format (line_numbers,  total_lines , sentences , characters)
      - label_encoder : the label encoder used to encode the classes

    Returns:
    --------
      Prints out the predicted label and the corresponding sequence/ text
  '''

  # Setup english sentence parser
  nlp = English()

  # Create sentence splitting pipeline object
  sentencizer = nlp.add_pipe('sentencizer')

  # Create doc of parsed sequences
  doc = nlp(abstract_dict['abstract'])

  # Return detected sentences from doc in string typpe
  abstract_lines = [str(sent) for sent in list(doc.sents)]

  # Get total number of lines
  total_lines_in_sample = len(abstract_lines)

  # Loop through each line in the abstract and create a list of dictionaries containing features
  sample_lines = []
  for i , line in enumerate(abstract_lines):
    sample_dict = {}
    sample_dict['text'] = str(line)
    sample_dict['line_number'] = i
    sample_dict['total_lines'] = total_lines_in_sample - 1
    sample_lines.append(sample_dict)


  # Get all line number and total lines numbers then one hot encode them
  abstract_line_numbers = [line['line_number'] for line in sample_lines]
  abstract_total_lines = [line['total_lines'] for line in sample_lines]

  abstract_line_numbers_one_hot = tf.one_hot(abstract_line_numbers , depth = 15)
  abstract_total_lines_one_hot = tf.one_hot(abstract_total_lines , depth = 20)


  # Split the lines into characters
  abstract_chars = [" ".join(list(sentence)) for sentence in abstract_lines]

  # Making prediction on sample features
  abstract_pred_probs = model.predict(x = (abstract_line_numbers_one_hot,
                                           abstract_total_lines_one_hot ,
                                           tf.constant(abstract_lines) ,
                                           tf.constant(abstract_chars)))

  # Turn prediction probs to pred class
  abstract_preds = tf.argmax(abstract_pred_probs , axis = 1)

  # Prediction class integers into string class name
  abstract_pred_classes = [label_encoder.classes_[i] for i in abstract_preds]

  # Prints out the abstract lines and the predicted sequence labels
  for i , line in enumerate(abstract_lines):
    print(f'{abstract_pred_classes[i]}:  {line}\n')

In [135]:
for abstract in example_abstracts:
  predict_on_abstract(abstract, model_5, label_encoder)

1/1 [==============================] - 0s 157ms/step
OBJECTIVE:  This RCT examined the efficacy of a manualized social intervention for children with HFASDs.

METHODS:  Participants were randomly assigned to treatment or wait-list conditions.

METHODS:  Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language.

METHODS:  A response-cost program was applied to reduce problem behaviors and foster skills acquisition.

RESULTS:  Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures).

METHODS:  Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents.

RESULTS:  High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity.

RESULTS:  Standardized effect size estimates were primarily in the medium and large ran